In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [4]:
TRAIN = 0
PREDICT = 1

armar_dataset=TRAIN

In [5]:
df_postulantes_edu_original = pd.read_csv('Originales/fiuba_1_postulantes_educacion.csv')
df_postulantes_datos_original = pd.read_csv('Originales/fiuba_2_postulantes_genero_y_edad.csv')
df_postulantes_vistas_original = pd.read_csv('Originales/fiuba_3_vistas.csv')
df_postulaciones_original = pd.read_csv('Originales/fiuba_4_postulaciones.csv')
df_avisos_online_original = pd.read_csv('Originales/fiuba_5_avisos_online.csv')
df_avisos_detalles_original = pd.read_csv('Originales/fiuba_6_avisos_detalle.csv')

In [6]:
df_postulantes_edu_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_1_postulantes_educacion.csv')
df_postulantes_datos_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_2_postulantes_genero_y_edad.csv')
df_postulantes_vistas_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_3_vistas.csv')
df_postulaciones_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_4_postulaciones.csv')
df_avisos_online_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_5_avisos_online.csv')
df_avisos_detalles_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_6_avisos_detalle.csv')

In [7]:
if (armar_dataset == PREDICT):
    df_postulantes_edu_desde = pd.read_csv('Desde 15 abril/fiuba_1_postulantes_educacion.csv')
    df_postulantes_datos_desde = pd.read_csv('Desde 15 abril/fiuba_2_postulantes_genero_y_edad.csv')
    df_postulantes_vistas_desde = pd.read_csv('Desde 15 abril/fiuba_3_vistas.csv')
    df_avisos_detalles_desde = pd.read_csv('Desde 15 abril/fiuba_6_avisos_detalle.csv')

In [8]:
df_avisos_detalles_faltantes = pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

# Detalles de avisos

Veamos el tamaño de los detalles de avisos para generar un dataset para entrenar

In [5]:
df_avisos_detalles_original.shape

(13534, 11)

In [6]:
df_avisos_detalles_nuevo.shape

(18299, 11)

In [8]:
df_avisos_detalles_faltantes.shape

(338, 11)

Juntamos todo

In [9]:
if (armar_dataset == PREDICT):
    df_avisos_detalles = pd.concat([df_avisos_detalles_original, df_avisos_detalles_nuevo, df_avisos_detalles_desde, df_avisos_detalles_faltantes])
else:
    df_avisos_detalles = pd.concat([df_avisos_detalles_original, df_avisos_detalles_nuevo, df_avisos_detalles_faltantes])

In [6]:
df_avisos_detalles.shape

(32171, 11)

Eliminamos duplicados (No tiene sentido que hayan avisos exactamente iguales, es decir, mismo Id de aviso)

In [6]:
df_avisos_detalles = df_avisos_detalles.drop_duplicates(subset='idaviso')

In [8]:
df_avisos_detalles.shape

(25288, 11)

Veamos las columnas utiles

In [9]:
df_avisos_detalles.columns

Index(['idaviso', 'idpais', 'titulo', 'descripcion', 'nombre_zona', 'ciudad',
       'mapacalle', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',
       'denominacion_empresa'],
      dtype='object')

Descartamos las que tengan mayoria de valors nulos, no aportan información relevante

In [13]:
df_avisos_detalles['mapacalle'].isnull().sum()

18052

In [14]:
df_avisos_detalles['ciudad'].isnull().sum()

19598

In [7]:
del df_avisos_detalles['mapacalle']
del df_avisos_detalles['ciudad']

El pais es único y está en todos los datos, no aporta nada

In [16]:
df_avisos_detalles['idpais'].nunique()

1

In [17]:
df_avisos_detalles['idpais'].isnull().sum()

0

In [8]:
del df_avisos_detalles['idpais']

In [19]:
df_avisos_detalles['nombre_zona'].isnull().sum()

0

In [20]:
df_avisos_detalles['nombre_zona'].value_counts()

Gran Buenos Aires              18022
Capital Federal                 1645
Buenos Aires (fuera de GBA)       17
GBA Oeste                         10
Corrientes                         1
Cordoba                            1
Name: nombre_zona, dtype: int64

In [9]:
del df_avisos_detalles['nombre_zona']

Nos quedamos con

In [22]:
df_avisos_detalles.columns

Index(['idaviso', 'titulo', 'descripcion', 'tipo_de_trabajo', 'nivel_laboral',
       'nombre_area', 'denominacion_empresa'],
      dtype='object')

Parseamos el titulo y la descripción para sacar datos relevantes y generar nuevos features

In [10]:
def lowercase_descripcion(row):
    if pd.isnull(row['descripcion']):
        return row['descripcion']   
    
    return row['descripcion'].lower()

def lowercase_titulo(row):
    if pd.isnull(row['titulo']):
        return row['titulo']   
    
    return row['titulo'].lower()

In [11]:
df_avisos_detalles['descripcion'] = df_avisos_detalles.apply(lowercase_descripcion, axis=1)
df_avisos_detalles['titulo'] = df_avisos_detalles.apply(lowercase_titulo, axis=1)

In [15]:
df_avisos_detalles.head()

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,vendedor/a provincia de santa fe,<p><strong><strong>empresa:</strong></strong> ...,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,enfermeras,<p>solicitamos para importante cadena de farma...,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,chofer de taxi,<p>te gusta manejar? queres ganar plata hacien...,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,chofer de camioneta bahia blanca - punta alta,<p><strong>somos una empresa multinacional que...,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,operarios de planta - rubro electrodomésticos,<p><strong>operarios de planta</strong></p><p>...,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [12]:
def extra(row, list_of_words):
    if pd.isnull(row['descripcion']) and pd.isnull(row['titulo']):
        return 0
    for word in list_of_words:
        if word in row['descripcion'] or word in row['titulo']:
            return 1
    return 0

In [13]:
pal_claves = {
    'web' : ['web', 'pagina', 'página'],
    'crecimiento' : ['crecim', 'progre'],
    'capacitacion' : ['aprender', 'aprendiza', 'capacitar', 'capacitacion', 'capacitación'],
    'clima laboral' : ['excelente cl', 'clima'],
    'multinacional' : ['multin', 'internacional'],
    'incorp_inmediata' : ['incorporación', 'incorporacion'],
    'disponibilidad_horaria' : ['fines de sem', 'sábado', 'sabado', 'domingo', 'feriado', 'rotativo'],
    'disponibilidad_viajar' : ['viaje', 'viajar'],
    'flexibilidad' : ['flexi'],
    'secundario' : ['secundario compl'],
    'terciario' : ['terciario'],
    'ingles' : ['inglés', 'ingles'],
    'estudiantes' : ['universi', 'estudiant'],
    'conoc_informatica' : ['excel', 'word', 'office'],
    'experiencia' : ['años de exp', 'experiencia'],
    'proactivo' : ['proactiv'],
    'relaciones_pers' : ['relaciones', 'personales'],
    'desarrollador' : ['desarr'],
    'cajero' : ['cajero'],
    'vendedor' : ['vended'],
    'ingeniero' : ['ingeni'],
    'analista' : ['analis'],
    'administrativo' : ['administra'],
    'junior' : ['jr', 'junior'],
    'jefe' : ['jefe'],
    'marketing' : ['marke', 'tele'],
    'supervisor' : ['superv']
}

for k, v in pal_claves.items():
    df_avisos_detalles[k] = df_avisos_detalles.apply((lambda x: extra(x, v)), axis=1)

In [14]:
del df_avisos_detalles['descripcion']
del df_avisos_detalles['titulo']

In [17]:
df_avisos_detalles.shape

(19696, 32)

In [18]:
df_avisos_detalles.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,desarrollador,cajero,vendedor,ingeniero,analista,administrativo,junior,jefe,marketing,supervisor
0,8725750,Full-time,Senior / Semi-Senior,Comercial,VENTOR,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,17903700,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000150677,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000610287,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1000872556,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
list_tipos_trabajos = []
list_nivel_laboral = []
list_nombre_area = []
list_empresa = []


def tipos_trabajos(row):
    if pd.isnull(row['tipo_de_trabajo']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['tipo_de_trabajo'] in list_tipos_trabajos:
        return list_tipos_trabajos.index(row['tipo_de_trabajo'])+1
    
    list_tipos_trabajos.append(row['tipo_de_trabajo'])
    
    return list_tipos_trabajos.index(row['tipo_de_trabajo'])+1

def nivel_laboral(row):
    if pd.isnull(row['nivel_laboral']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['nivel_laboral'] in list_nivel_laboral:
        return list_nivel_laboral.index(row['nivel_laboral'])+1
    
    list_nivel_laboral.append(row['nivel_laboral'])
    
    return list_nivel_laboral.index(row['nivel_laboral'])+1

def nombre_area(row):
    if pd.isnull(row['nombre_area']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['nombre_area'] in list_nombre_area:
        return list_nombre_area.index(row['nombre_area'])+1
    
    list_nombre_area.append(row['nombre_area'])
    
    return list_nombre_area.index(row['nombre_area'])+1

def nombre_empresa(row):
    if pd.isnull(row['denominacion_empresa']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['denominacion_empresa'] in list_empresa:
        return list_empresa.index(row['denominacion_empresa'])+1
    
    list_empresa.append(row['denominacion_empresa'])
    
    return list_empresa.index(row['denominacion_empresa'])+1

df_avisos_detalles['tipo_de_trabajo'] = df_avisos_detalles.apply(tipos_trabajos, axis=1)
df_avisos_detalles['nivel_laboral'] = df_avisos_detalles.apply(nivel_laboral, axis=1)
df_avisos_detalles['nombre_area'] = df_avisos_detalles.apply(nombre_area, axis=1)
df_avisos_detalles['denominacion_empresa'] = df_avisos_detalles.apply(nombre_empresa, axis=1)

In [20]:
df_avisos_detalles.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,desarrollador,cajero,vendedor,ingeniero,analista,administrativo,junior,jefe,marketing,supervisor
0,8725750,1,1,1,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,17903700,1,1,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000150677,1,1,3,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000610287,1,1,3,4,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1000872556,1,1,4,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
df_avisos_detalles.shape

(19696, 32)

In [16]:
df_avisos_online = pd.concat([df_avisos_online_original, df_avisos_online_nuevo])
df_avisos_online = df_avisos_online.drop_duplicates()

In [17]:
df_avisos_online['online'] = 1
df_avisos_detalles = pd.merge(df_avisos_detalles, df_avisos_online, how='left', on=['idaviso'])

In [18]:
df_avisos_detalles['online'] = df_avisos_detalles['online'].fillna(value=0)

In [19]:
df_avisos_detalles.shape

(19696, 33)

Usar postulaciones para rellenar vistas que no tenga


In [19]:
postulaciones = pd.concat([df_postulaciones_original, df_postulaciones_nuevo])

In [20]:
postulaciones.shape

(8311264, 3)

In [21]:
postulaciones = postulaciones.drop_duplicates(['idaviso', 'idpostulante'])

In [23]:
postulaciones.shape

(6603752, 3)

In [22]:
postulaciones_total = postulaciones.loc[:, ['idaviso', 'idpostulante']].rename(columns={'idpostulante':'postu_total'})\
                .groupby('idaviso').count().reset_index()

In [23]:
df_avisos_detalles = pd.merge(df_avisos_detalles, postulaciones_total, on=['idaviso'], how='left')

In [26]:
df_avisos_detalles.shape

(25288, 34)

In [24]:
if (armar_dataset == PREDICT):
    vistas = pd.concat([df_postulantes_vistas_original, df_postulantes_vistas_nuevo, df_postulantes_vistas_desde])
else:
    vistas = pd.concat([df_postulantes_vistas_original, df_postulantes_vistas_nuevo])

In [25]:
vistas.shape

(6720583, 3)

In [26]:
vistas = vistas.drop_duplicates()

In [33]:
vistas.shape

(5073861, 3)

In [27]:
vistas = vistas.rename(columns={'idAviso' : 'idaviso'})
vistas.shape

(6682624, 3)

In [28]:
vistas2 = vistas.loc[:, ['idaviso', 'idpostulante']].groupby('idaviso').count().reset_index()

In [29]:
vistas2 = vistas2.rename(columns={'idpostulante' : 'total_vistas'})

In [30]:
df_avisos_detalles = pd.merge(df_avisos_detalles, vistas2, how='left', on=['idaviso'])

In [39]:
df_avisos_detalles.head()

,idaviso,idpais,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,...,ingeniero,analista,administrativo,junior,jefe,marketing,supervisor,online,postu_total,total_vistas
0,8725750,1,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0.0,37.0,NaN
1,17903700,1,1,1,2,2,0,0,0,0,...,0,0,0,0,0,0,0,1.0,47.0,1166.0
2,1000150677,1,1,1,3,3,0,0,0,0,...,0,0,0,0,0,0,0,0.0,NaN,NaN
3,1000610287,1,1,1,3,4,0,1,1,0,...,0,0,0,0,0,0,0,1.0,45.0,50.0
4,1000872556,1,1,1,4,5,0,0,0,0,...,0,0,0,0,0,0,1,0.0,NaN,NaN


In [31]:
vistas2 = vistas.groupby(['idaviso', 'idpostulante']).count().reset_index()

In [32]:
vistas2 = vistas2.loc[:, ['idaviso', 'timestamp']].groupby('idaviso').median().reset_index()

In [33]:
vistas2 = vistas2.rename(columns={'timestamp' : 'vistas_median'})

In [34]:
df_avisos_detalles = pd.merge(df_avisos_detalles, vistas2, how='left', on=['idaviso'])

In [44]:
df_avisos_detalles.head()

,idaviso,idpais,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,...,analista,administrativo,junior,jefe,marketing,supervisor,online,postu_total,total_vistas,vistas_median
0,8725750,1,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0.0,37.0,NaN,NaN
1,17903700,1,1,1,2,2,0,0,0,0,...,0,0,0,0,0,0,1.0,47.0,1166.0,2.0
2,1000150677,1,1,1,3,3,0,0,0,0,...,0,0,0,0,0,0,0.0,NaN,NaN,NaN
3,1000610287,1,1,1,3,4,0,1,1,0,...,0,0,0,0,0,0,1.0,45.0,50.0,2.0
4,1000872556,1,1,1,4,5,0,0,0,0,...,0,0,0,0,0,1,0.0,NaN,NaN,NaN


In [35]:
vistas["timestamp"] = pd.to_datetime(vistas["timestamp"])

In [36]:
postulaciones['fechapostulacion'] = pd.to_datetime(postulaciones["fechapostulacion"], errors = 'coerce')

In [37]:
fechas_postulaciones = postulaciones.loc[:, ['idaviso', 'fechapostulacion']].groupby('idaviso').min().reset_index()

In [38]:
vistas = vistas.rename(columns={'timestamp' : 'fechapostulacion'})

In [39]:
fechas_vistas_postul = vistas.loc[:, ['idaviso', 'fechapostulacion']].groupby('idaviso').min().reset_index()

In [40]:
fechas_iniciales = pd.concat([fechas_postulaciones, fechas_vistas_postul])

In [41]:
fechas_iniciales.shape

(35266, 2)

In [42]:
fechas_iniciales = fechas_iniciales.groupby('idaviso').min().reset_index()

In [43]:
fechas_iniciales.shape

(21158, 2)

In [44]:
fechas_iniciales = fechas_iniciales.rename(columns={'fechapostulacion' : 'fecha_inicio'})

In [45]:
fechas_postulaciones = postulaciones.loc[:, ['idaviso', 'fechapostulacion']].groupby('idaviso').max().reset_index()

In [46]:
fechas_vistas_postul = vistas.loc[:, ['idaviso', 'fechapostulacion']].groupby('idaviso').max().reset_index()

In [47]:
fechas_finales = pd.concat([fechas_postulaciones, fechas_vistas_postul])

In [48]:
fechas_finales.shape

(35266, 2)

In [49]:
fechas_finales = fechas_finales.groupby('idaviso').max().reset_index()

In [50]:
fechas_finales.shape

(21158, 2)

In [51]:
fechas_finales = fechas_finales.rename(columns={'fechapostulacion':'fecha_fin'})

In [52]:
fechas_totales = pd.merge(fechas_finales, fechas_iniciales, on=['idaviso'], how='inner')

In [53]:
fechas_totales.shape

(21158, 3)

In [64]:
fechas_totales.head()

,idaviso,fecha_fin,fecha_inicio
0,8725750,2018-02-08 10:49:43.000,2018-01-15 05:30:03.000
1,11740890,2018-04-11 21:15:46.000,2018-01-16 08:43:25.000
2,12543760,2018-04-16 15:27:54.016,2018-01-18 07:56:55.000
3,12812680,2018-04-09 18:32:49.226,2018-02-20 14:12:52.000
4,17903700,2018-04-17 08:34:24.000,2018-02-24 17:44:47.891


In [54]:
def set_dias(row):
    return row.days

In [55]:
def dia_anio(row):
    return row.dayofyear

def dia_semana(row):
    return row.dayofweek

In [56]:
segs_day = 86400

def cos_hora(row):
    return np.cos(2*np.pi*(row.hour*60*60 + row.minute*60 + row.second)/segs_day)

def sin_hora(row):
    return np.sin(2*np.pi*(row.hour*60*60 + row.minute*60 + row.second)/segs_day)

In [57]:
def is_month_start(row):
    return row.is_month_start

def is_month_end(row):
    return row.is_month_end

In [58]:
fechas_totales['cant_dias'] = (fechas_totales['fecha_fin']-fechas_totales['fecha_inicio']).apply(set_dias)

In [59]:
fechas_totales['dia_semana_inicio'] = (fechas_totales['fecha_inicio']).apply(dia_semana)

In [60]:
fechas_totales['dia_anio_inicio'] = (fechas_totales['fecha_inicio']).apply(dia_anio)

In [61]:
fechas_totales['dia_anio_fin'] = (fechas_totales['fecha_fin']).apply(dia_anio)

In [62]:
fechas_totales['hora_inicio_cos'] = (fechas_totales['fecha_inicio']).apply(cos_hora)

In [63]:
fechas_totales['hora_inicio_sen'] = (fechas_totales['fecha_inicio']).apply(sin_hora)

In [64]:
fechas_totales['hora_fin_cos'] = (fechas_totales['fecha_fin']).apply(cos_hora)

In [65]:
fechas_totales['hora_fin_sen'] = (fechas_totales['fecha_fin']).apply(sin_hora)

In [66]:
fechas_totales['inicio_mes'] = (fechas_totales['fecha_inicio']).apply(is_month_start)

In [67]:
fechas_totales['fin_mes'] = (fechas_totales['fecha_inicio']).apply(is_month_end)

In [82]:
fechas_totales.head()

,idaviso,fecha_fin,fecha_inicio,cant_dias,dia_semana_inicio,dia_anio_inicio,dia_anio_fin,hora_inicio_cos,hora_inicio_sen,hora_fin_cos,hora_fin_sen,inicio_mes,fin_mes
0,8725750,2018-02-08 10:49:43.000,2018-01-15 05:30:03.000,24,0,15,39,0.130310,0.991473,-0.953344,0.301885,False,False
1,11740890,2018-04-11 21:15:46.000,2018-01-16 08:43:25.000,85,1,16,101,-0.654136,0.756377,0.754041,-0.656827,False,False
2,12543760,2018-04-16 15:27:54.016,2018-01-18 07:56:55.000,88,3,18,106,-0.488304,0.872674,-0.616005,-0.787742,False,False
3,12812680,2018-04-09 18:32:49.226,2018-02-20 14:12:52.000,48,1,51,99,-0.836605,-0.547807,0.142701,-0.989766,False,False
4,17903700,2018-04-17 08:34:24.000,2018-02-24 17:44:47.891,51,5,55,107,-0.066346,-0.997797,-0.623880,0.781520,False,False


In [65]:
fechas_totales.head()

,idaviso,fecha_fin,fecha_inicio,cant_dias
0,8725750,2018-02-09 15:15:25.000,2018-01-15 05:30:03.000,25
1,11740890,2018-03-25 11:03:26.000,2018-01-27 13:41:07.000,56
2,12543760,2018-04-09 01:43:46.442,2018-01-18 08:00:03.000,80
3,12812680,2018-04-09 18:32:49.226,2018-02-20 14:12:52.000,48
4,17903700,2018-04-17 08:34:24.000,2018-02-24 17:44:47.891,51


In [68]:
del fechas_totales['fecha_fin']
del fechas_totales['fecha_inicio']

In [69]:
df_avisos_detalles.shape

(19696, 36)

In [70]:
df_avisos_detalles_final = pd.merge(df_avisos_detalles, fechas_totales, on=['idaviso'], how='left')

In [88]:
df_avisos_detalles_final.shape

(19696, 47)

In [89]:
df_avisos_detalles_final.head()

,idaviso,idpais,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,...,cant_dias,dia_semana_inicio,dia_anio_inicio,dia_anio_fin,hora_inicio_cos,hora_inicio_sen,hora_fin_cos,hora_fin_sen,inicio_mes,fin_mes
0,8725750,1,1,1,1,1,0,0,1,0,...,24.0,0.0,15.0,39.0,0.130310,0.991473,-0.953344,0.301885,False,False
1,17903700,1,1,1,2,2,0,0,0,0,...,51.0,5.0,55.0,107.0,-0.066346,-0.997797,-0.623880,0.781520,False,False
2,1000150677,1,1,1,3,3,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000610287,1,1,1,3,4,0,1,1,0,...,83.0,0.0,15.0,99.0,-0.915575,0.402148,0.918734,0.394877,False,False
4,1000872556,1,1,1,4,5,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
df_avisos_detalles_final.isnull().sum()

idaviso                      0
idpais                       0
tipo_de_trabajo              0
nivel_laboral                0
nombre_area                  0
denominacion_empresa         0
web                          0
crecimiento                  0
capacitacion                 0
clima laboral                0
multinacional                0
incorp_inmediata             0
disponibilidad_horaria       0
disponibilidad_viajar        0
flexibilidad                 0
secundario                   0
terciario                    0
ingles                       0
estudiantes                  0
conoc_informatica            0
experiencia                  0
proactivo                    0
relaciones_pers              0
desarrollador                0
cajero                       0
vendedor                     0
ingeniero                    0
analista                     0
administrativo               0
junior                       0
jefe                         0
marketing                    0
supervis

In [71]:
a = df_avisos_detalles_final['vistas_median'].mean()
a

1.3208846025939474

In [72]:
def fill_total_vistas(row):
    if((np.isnan(row['total_vistas'])) & (np.isnan(row['vistas_median']) == False) & (np.isnan(row['postu_total']) == False)):
        return row['postu_total']*row['vistas_median']
    elif((np.isnan(row['total_vistas'])) & (np.isnan((row['vistas_median']))) & (np.isnan((row['postu_total']))==False)):
        return row['postu_total']*a
    elif((np.isnan(row['total_vistas']) == False) & (np.isnan(row['postu_total']) == False) & (np.isnan(row['vistas_median']) == False)):
        if(row['total_vistas']<(row['postu_total']*row['vistas_median'])):
            return row['postu_total']*row['vistas_median']
    elif((np.isnan(row['total_vistas']) == False)&(np.isnan(row['postu_total']) == False)&(np.isnan(row['vistas_median']))):
        if(row['total_vistas']<row['postu_total']):
            return row['postu_total']
        
    return row['total_vistas']

In [73]:
df_avisos_detalles_final['total_vistas'] = df_avisos_detalles_final.apply(fill_total_vistas, axis=1)

In [94]:
df_avisos_detalles_final.isnull().sum()

idaviso                      0
idpais                       0
tipo_de_trabajo              0
nivel_laboral                0
nombre_area                  0
denominacion_empresa         0
web                          0
crecimiento                  0
capacitacion                 0
clima laboral                0
multinacional                0
incorp_inmediata             0
disponibilidad_horaria       0
disponibilidad_viajar        0
flexibilidad                 0
secundario                   0
terciario                    0
ingles                       0
estudiantes                  0
conoc_informatica            0
experiencia                  0
proactivo                    0
relaciones_pers              0
desarrollador                0
cajero                       0
vendedor                     0
ingeniero                    0
analista                     0
administrativo               0
junior                       0
jefe                         0
marketing                    0
supervis

In [76]:
df_avisos_detalles_final.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,administrativo,junior,jefe,marketing,supervisor,online,postu_total,total_vistas,vistas_median,cant_dias
0,8725750,1,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0.0,31.0,31.0,NaN,26.0
1,17903700,1,1,2,2,0,0,0,0,0,...,0,0,0,0,0,1.0,38.0,1337.0,2.0,51.0
2,1000150677,1,1,3,3,0,0,0,0,0,...,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN
3,1000610287,1,1,3,4,0,1,1,0,1,...,0,0,0,0,0,1.0,41.0,82.0,2.0,51.0
4,1000872556,1,1,4,5,0,0,0,0,0,...,0,0,0,0,1,0.0,NaN,1.0,1.0,0.0


In [74]:
if (armar_dataset == PREDICT):
    df_avisos_detalles_final.to_csv('Predecir/avisos_detalles_predict.csv')
else:
    df_avisos_detalles_final.to_csv('Predecir/avisos_detalles_train.csv')

# Postulantes

In [75]:
if (armar_dataset == PREDICT):
    df_postulantes_edu = pd.concat([df_postulantes_edu_original, df_postulantes_edu_nuevo, df_postulantes_edu_desde])
else:
    df_postulantes_edu = pd.concat([df_postulantes_edu_original, df_postulantes_edu_nuevo])

In [97]:
df_postulantes_edu.shape

(705684, 3)

In [98]:
df_postulantes_edu.columns

Index(['idpostulante', 'nombre', 'estado'], dtype='object')

In [99]:
df_postulantes_edu.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [100]:
df_postulantes_edu['nombre'].value_counts()

Universitario        259550
Secundario           246012
Terciario/Técnico    114815
Otro                  53513
Posgrado              20885
Master                10297
Doctorado               612
Name: nombre, dtype: int64

In [76]:
df_postulantes_edu = df_postulantes_edu.drop_duplicates()

In [79]:
df_postulantes_edu.shape

(693641, 3)

In [77]:
nivel_dict = {'Universitario' : 3
             , 'Secundario' : 1
             , 'Terciario/Técnico' : 2
             , 'Otro': 0
             , 'Posgrado' : 4
             , 'Master' : 5
             , 'Doctorado' : 6}

def educacion(row):
    return nivel_dict[row['nombre']]

df_postulantes_edu['nombre'] = df_postulantes_edu.apply(educacion, axis=1)

In [78]:
df_postulantes_edu['estado'].value_counts()

Graduado      370213
En Curso      149388
Abandonado     49011
Name: estado, dtype: int64

In [79]:
estado_dict = {'Graduado' : 2
             , 'En Curso' : 1
             , 'Abandonado' : 0}

def estado(row):
    return estado_dict[row['estado']]

df_postulantes_edu['estado'] = df_postulantes_edu.apply(estado, axis=1)

In [91]:
df_postulantes_edu.shape

(568612, 3)

In [92]:
df_postulantes_edu['idpostulante'].nunique()

366395

Nos quedamos unicamente con su mayor nivel de estudio

In [80]:
for_join = df_postulantes_edu.loc[:, ['idpostulante', 'nombre']].groupby('idpostulante').max().reset_index()
df_postulantes_edu = pd.merge(df_postulantes_edu, for_join, how='inner', on=['idpostulante', 'nombre'])

In [81]:
df_postulantes_edu.shape

(387698, 3)

In [82]:
df_postulantes_edu['idpostulante'].nunique()

366395

In [83]:
df_postulantes_edu.head()

,idpostulante,nombre,estado
0,NdJl,4,1
1,1d2B,3,1
2,NPBx,5,1
3,ZKl5,5,1
4,Y043Z,3,1


In [84]:
for_join = df_postulantes_edu.loc[:, ['idpostulante', 'estado']].groupby('idpostulante').max().reset_index()
df_postulantes_edu = pd.merge(df_postulantes_edu, for_join, how='inner', on=['idpostulante', 'estado'])

In [108]:
df_postulantes_edu['idpostulante'].nunique()

366395

In [85]:
if (armar_dataset == PREDICT):
    df_postulantes_datos = pd.concat([df_postulantes_datos_original, df_postulantes_datos_nuevo, df_postulantes_datos_desde])
else:
    df_postulantes_datos = pd.concat([df_postulantes_datos_original, df_postulantes_datos_nuevo])
    
df_postulantes_datos = df_postulantes_datos.drop_duplicates()

In [100]:
df_postulantes_datos.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [101]:
df_postulantes_datos['idpostulante'].nunique()

408146

In [86]:
df_postulantes_datos['edad'] = pd.to_datetime(df_postulantes_datos["fechanacimiento"], errors = 'coerce')
del df_postulantes_datos['fechanacimiento']

In [87]:
df_postulantes_datos['idpostulante'].nunique()

408146

In [88]:
list_sexos = []

def tipo_sexos(row):
    if pd.isnull(row['sexo']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['sexo'] in list_sexos:
        return list_sexos.index(row['sexo'])+1
    
    list_sexos.append(row['sexo'])
    
    return list_sexos.index(row['sexo'])+1

df_postulantes_datos['sexo'] = df_postulantes_datos.apply(tipo_sexos, axis=1)

In [89]:
def edad(row):
    return 2018-row['edad'].year

df_postulantes_datos['edad'] = df_postulantes_datos.apply(edad, axis=1)

In [90]:
df_postulantes_datos.shape

(408452, 3)

In [91]:
df_postulantes_datos = df_postulantes_datos.drop_duplicates('idpostulante')

In [108]:
df_postulantes_datos.shape

(408146, 3)

In [109]:
df_postulantes_datos['idpostulante'].nunique()

408146

In [110]:
df_postulantes_edu['idpostulante'].nunique()

366395

In [92]:
df_postulantes = pd.merge(df_postulantes_edu, df_postulantes_datos, how='outer', on=['idpostulante'])

In [93]:
df_postulantes['idpostulante'].nunique()

408146

In [94]:
df_postulantes = df_postulantes.drop_duplicates('idpostulante')

In [114]:
df_postulantes.head()

,idpostulante,nombre,estado,sexo,edad
0,NdJl,4.0,1.0,2,49.0
1,1d2B,3.0,1.0,2,42.0
2,NPBx,5.0,1.0,2,37.0
3,ZKl5,5.0,1.0,2,44.0
4,Y043Z,3.0,1.0,2,39.0


In [115]:
df_postulantes.shape

(408146, 5)

In [116]:
vistas.head()

,idaviso,fechapostulacion,idpostulante
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP


In [95]:
vistas2 = vistas.loc[:, ['idaviso', 'idpostulante']].groupby('idpostulante').count().reset_index()
vistas2 = vistas2.rename(columns={'idaviso' : 'vistas_postulante'})

In [96]:
df_postulantes = pd.merge(df_postulantes, vistas2, on=['idpostulante'], how='left')

In [96]:
df_postulantes.head()

,idpostulante,nombre,estado,sexo,edad,vistas_postulante
0,NdJl,4.0,1.0,2,49.0,60.0
1,1d2B,3.0,1.0,2,42.0,NaN
2,NPBx,5.0,1.0,2,37.0,16.0
3,ZKl5,5.0,1.0,2,44.0,41.0
4,Y043Z,3.0,1.0,2,39.0,NaN


Considerar agregar vistas por dia!

In [97]:
fechas_postulaciones = postulaciones.loc[:, ['idpostulante', 'fechapostulacion']].groupby('idpostulante').min().reset_index()

In [98]:
vistas = vistas.rename(columns={'timestamp' : 'fechapostulacion'})

In [99]:
fechas_vistas_postul = vistas.loc[:, ['idpostulante', 'fechapostulacion']].groupby('idpostulante').min().reset_index()

In [100]:
fechas_iniciales = pd.concat([fechas_postulaciones, fechas_vistas_postul])

In [101]:
fechas_iniciales.shape

(655087, 2)

In [102]:
fechas_iniciales = fechas_iniciales.groupby('idpostulante').min().reset_index()

In [103]:
fechas_iniciales.shape

(393425, 2)

In [104]:
fechas_iniciales = fechas_iniciales.rename(columns={'fechapostulacion' : 'fecha_inicio_actividad'})

In [105]:
fechas_postulaciones = postulaciones.loc[:, ['idpostulante', 'fechapostulacion']].groupby('idpostulante').max().reset_index()

In [106]:
fechas_vistas_postul = vistas.loc[:, ['idpostulante', 'fechapostulacion']].groupby('idpostulante').max().reset_index()

In [107]:
fechas_finales = pd.concat([fechas_postulaciones, fechas_vistas_postul])

In [108]:
fechas_finales.shape

(655087, 2)

In [109]:
fechas_finales = fechas_finales.groupby('idpostulante').max().reset_index()

In [110]:
fechas_finales.shape

(393425, 2)

In [111]:
fechas_finales = fechas_finales.rename(columns={'fechapostulacion' : 'fecha_fin_actividad'})

In [112]:
fechas_totales = pd.merge(fechas_finales, fechas_iniciales, on=['idpostulante'], how='inner')

In [110]:
fechas_totales.head()

,idpostulante,fecha_fin_actividad,fecha_inicio_actividad
0,0002q,2018-04-29 13:59:51.366,2018-04-29 13:41:52.438
1,0005E,2018-04-27 09:31:30.321,2018-04-22 17:17:57.744
2,000R8,2018-04-24 07:21:17.140,2018-04-24 02:38:12.731
3,001XE,2018-04-30 17:22:06.703,2018-04-17 11:49:36.040
4,003k9,2018-04-25 17:05:47.018,2018-04-16 21:35:18.819


In [113]:
fechas_totales['cant_dias_actividad'] = (fechas_totales['fecha_fin_actividad']-fechas_totales['fecha_inicio_actividad']).apply(set_dias)

In [114]:
fechas_totales['dia_inicio_actividad'] = fechas_totales['fecha_inicio_actividad'].apply(dia_anio)

In [115]:
fechas_totales['dia_fin_actividad'] = fechas_totales['fecha_fin_actividad'].apply(dia_anio)

In [114]:
fechas_totales.head()

,idpostulante,fecha_fin_actividad,fecha_inicio_actividad,cant_dias_actividad,dia_inicio_actividad,dia_fin_actividad
0,0002q,2018-04-29 13:59:51.366,2018-04-29 13:41:52.438,0,119,119
1,0005E,2018-04-27 09:31:30.321,2018-04-22 17:17:57.744,4,112,117
2,000R8,2018-04-24 07:21:17.140,2018-04-24 02:38:12.731,0,114,114
3,001XE,2018-04-30 17:22:06.703,2018-04-17 11:49:36.040,13,107,120
4,003k9,2018-04-25 17:05:47.018,2018-04-16 21:35:18.819,8,106,115


In [116]:
del fechas_totales['fecha_fin_actividad']
del fechas_totales['fecha_inicio_actividad']

In [117]:
df_postulantes = pd.merge(df_postulantes, fechas_totales, on=['idpostulante'], how='left')

In [139]:
df_postulantes.shape

(408146, 9)

Agregamos tipo de aviso más popular por postulante, empresa popular y nombre de area popular

In [140]:
df_avisos_detalles_final.head()

,idaviso,idpais,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,...,cant_dias,dia_semana_inicio,dia_anio_inicio,dia_anio_fin,hora_inicio_cos,hora_inicio_sen,hora_fin_cos,hora_fin_sen,inicio_mes,fin_mes
0,8725750,1,1,1,1,1,0,0,1,0,...,24.0,0.0,15.0,39.0,0.130310,0.991473,-0.953344,0.301885,False,False
1,17903700,1,1,1,2,2,0,0,0,0,...,51.0,5.0,55.0,107.0,-0.066346,-0.997797,-0.623880,0.781520,False,False
2,1000150677,1,1,1,3,3,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000610287,1,1,1,3,4,0,1,1,0,...,83.0,0.0,15.0,99.0,-0.915575,0.402148,0.918734,0.394877,False,False
4,1000872556,1,1,1,4,5,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
postulaciones_total = postulaciones.loc[:, ['idaviso', 'idpostulante']].rename(columns={'idaviso':'postu_postulante'})\
                .groupby('idpostulante').count().reset_index()

In [120]:
df_postulantes = pd.merge(df_postulantes, postulaciones_total, on=['idpostulante'], how='left')

In [121]:
def fill_vistas_postu(row):
    if((np.isnan(row['vistas_postulante'])) & (np.isnan(row['postu_postulante']) == False)):
        return row['postu_postulante']
    elif((np.isnan(row['vistas_postulante']) == False) & (np.isnan(row['postu_postulante']) == False)):
        if(row['postu_postulante']>row['vistas_postulante']):
            return row['postu_postulante']
        
    return row['vistas_postulante']

In [122]:
df_postulantes['vistas_postulante'] = df_postulantes.apply(fill_vistas_postu, axis=1)

In [122]:
df_postulantes.head()

,idpostulante,nombre,estado,sexo,edad,vistas_postulante,cant_dias_actividad,dia_inicio_actividad,dia_fin_actividad,postu_postulante
0,NdJl,4.0,1.0,2,49.0,141.0,103.0,16.0,120.0,30.0
1,1d2B,3.0,1.0,2,42.0,4.0,7.0,50.0,57.0,4.0
2,NPBx,5.0,1.0,2,37.0,34.0,99.0,19.0,118.0,34.0
3,ZKl5,5.0,1.0,2,44.0,47.0,89.0,18.0,107.0,11.0
4,Y043Z,3.0,1.0,2,39.0,12.0,50.0,45.0,95.0,4.0


In [123]:
if (armar_dataset == PREDICT):
    df_postulantes.to_csv('Predecir/postulantes_datos_predict.csv')
else:
    df_postulantes.to_csv('Predecir/postulantes_datos_train.csv')

# Postulaciones con postulantes (To train)

In [124]:
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [122]:
del postulaciones['fechapostulacion']

In [123]:
postulaciones.to_csv('Predecir/postulaciones_train.csv')

In [124]:
if (armar_dataset == PREDICT):
    vistas.to_csv('Predecir/vistas_predict.csv')
else:
    vistas.to_csv('Predecir/vistas_train.csv')